# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vac_df=pd.read_csv('../output_data/weather_data.csv').drop('City_ID', axis=1)
vac_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Turan,16.07,108.22,93.20,52,40,8.05,VN,1596343972
1,Punta Arenas,-53.15,-70.92,35.60,93,100,14.99,CL,1596343889
2,Pangnirtung,66.15,-65.71,45.10,80,96,1.16,CA,1596343972
3,Ucluelet,48.93,-125.55,61.00,86,89,0.92,CA,1596343973
4,Port Macquarie,-31.43,152.92,69.01,64,0,1.01,AU,1596343973


In [3]:
len(vac_df)

587

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key)
locations = vac_df[['Lat', 'Lng']]
humidity=vac_df['Humidity']
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=4)
fig.add_layer(heat_layer)
plt.savefig('../output_data/heat_map.png')
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
vac_drill_df=vac_df.loc[(vac_df['Max Temp']>70) & (vac_df['Max Temp']<80)]
len(vac_drill_df)

169

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df=vac_drill_df.copy()
hotel_df['Hotel Name'] = ""
len(hotel_df)
hotel_df=hotel_df.reset_index()
hotel_df=hotel_df.drop('index', axis=1)

In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

target_radius = 5000
target_type = "hotel"
# set up a parameters dictionary
params = {
    "radius": target_radius,
    "keyword": target_type,
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f'{lat},{lng}'

    response=requests.get(base_url, params=params)
    places_data=response.json()
    try:
        hotel_df.loc[index, 'Hotel Name']=places_data['results'][0]['name']
        print(f"hotel name for {row['City']} is {places_data['results'][0]['name']}")
        
    except:
        print(f"Failed hotel lookup for {row['City']}")
  

Failed hotel lookup for Albany
hotel name for Lüderitz is LÜDERITZ NEST HOTEL
hotel name for Mogadishu is Dolphin Hotel
hotel name for Laguna is Hilton Garden Inn Sacramento Elk Grove
hotel name for Yulara is Sails in the Desert
hotel name for Talnakh is Talnakh Hotel
hotel name for Vaini is Paradise First Hotel
hotel name for Rikitea is Pension Bianca & Benoit
Failed hotel lookup for Sokolo
hotel name for Onda is Motel Shhh
hotel name for Yellowknife is The Explorer Hotel
hotel name for Emerald is Emerald Star Hotel
hotel name for Iquitos is DoubleTree by Hilton Iquitos
hotel name for Turukhansk is ООО "Туруханская пушнина"
hotel name for Diffa is Complexe Hôtelier Du 18 Décembre
Failed hotel lookup for Svetlaya
hotel name for Khartoum is Jori Vil Hotel
hotel name for Nacala is Nacala Plaza Business Design Hotel
hotel name for Hamilton is Hampton Inn & Suites Cincinnati/Uptown-University Area
hotel name for Saskylakh is Saskylakh Aeroport
Failed hotel lookup for Airai
Failed hotel loo

In [11]:
hotel_df.head()
hotel_df=hotel_df.loc[hotel_df['Hotel Name']!='']


In [12]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Lüderitz,-26.65,15.16,71.6,18,0,9.17,NaN,1596343974,LÜDERITZ NEST HOTEL
2,Mogadishu,2.04,45.34,75.2,69,20,17.22,SO,1596343707,Dolphin Hotel
3,Laguna,38.42,-121.42,78.8,39,1,4.70,US,1596343975,Hilton Garden Inn Sacramento Elk Grove
4,Yulara,-25.24,130.99,75.2,8,0,8.05,AU,1596343977,Sails in the Desert
5,Talnakh,69.49,88.40,71.6,40,84,15.66,RU,1596343977,Talnakh Hotel


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig=gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))